In [2]:
pip install pandas scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [3]:
! pip install accelerate -U
! pip install tokenizers
! pip install transformers datasets evaluate

   ---------------------------------------- 0.0/203.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/203.1 MB ? eta -:--:--
   ---------------------------------------- 0.5/203.1 MB 1.4 MB/s eta 0:02:26
   ---------------------------------------- 0.8/203.1 MB 1.5 MB/s eta 0:02:19
   ---------------------------------------- 1.0/203.1 MB 1.5 MB/s eta 0:02:17
   ---------------------------------------- 1.3/203.1 MB 1.3 MB/s eta 0:02:31
   ---------------------------------------- 1.8/203.1 MB 1.3 MB/s eta 0:02:30
   ---------------------------------------- 2.1/203.1 MB 1.3 MB/s eta 0:02:33
   ---------------------------------------- 2.4/203.1 MB 1.3 MB/s eta 0:02:41
    --------------------------------------- 2.6/203.1 MB 1.2 MB/s eta 0:02:46
    --------------------------------------- 2.6/203.1 MB 1.2 MB/s eta 0:02:46
    --------------------------------------- 2.9/203.1 MB 1.0 MB/s eta 0:03:16
    --------------------------------------- 3.1/203.1 MB 997.4 kB/s eta 0:03:2

In [4]:
import pandas as pd

poetry = pd.read_csv('data/merged_data.tsv', sep='\t', encoding='ISO-8859-1').drop(columns="no")

print(poetry)

                                                   text   label
0                       Klo fadli zon goblok, boleeeh?       CB
1     @detikcom Jangan nyinyirin pak fadli zon, dia ...      CB
2     @ahmadwaluy @bravo1282 @fadlizon Kritik siapa ...      CB
3      @fadlizon @jokowi Hati penuh dendam,  pikiran...      CB
4     @fadlizon @jokowi Kerja anda apa zon bisanya c...      CB
...                                                 ...     ...
1398                     babi itu sangat kotor dan bau.      CB
1399      anjing, hebat banget cara lu nyelesain tugas.      CB
1400           anjing itu sangat loyal pada pemiliknya.  Non_CB
1401                     monyet, lu selalu bikin ribet!      CB
1402   gue liat monyet bermain di hutan, sangat lincah.  Non_CB

[1403 rows x 2 columns]


In [5]:
import re
import string
import pandas as pd

def clean_text(text):
    # Menghapus URL
    cleaned_text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Menghapus mention (@username) dan hashtag (#hashtag)
    cleaned_text = re.sub(r'\@\w+|\#', '', cleaned_text)
    # Menghapus angka
    cleaned_text = re.sub(r'\d+', '', cleaned_text)
    # Menghapus tanda baca
    cleaned_text = cleaned_text.translate(str.maketrans('', '', string.punctuation))
    # Mengubah ke huruf kecil
    cleaned_text = cleaned_text.lower()
    return cleaned_text

poetry['cleaned_text'] = poetry['text'].apply(clean_text)

print(poetry[['text', 'cleaned_text']].head())


                                                text  \
0                    Klo fadli zon goblok, boleeeh?    
1  @detikcom Jangan nyinyirin pak fadli zon, dia ...   
2  @ahmadwaluy @bravo1282 @fadlizon Kritik siapa ...   
3   @fadlizon @jokowi Hati penuh dendam,  pikiran...   
4  @fadlizon @jokowi Kerja anda apa zon bisanya c...   

                                        cleaned_text  
0                      klo fadli zon goblok boleeeh   
1   jangan nyinyirin pak fadli zon dia gitu punya...  
2     kritik siapa bisa di penjara fadli zon krit...  
3     hati penuh dendam  pikiran selalu negatif m...  
4    kerja anda apa zon bisanya cuma nyinyi r aja...  


In [6]:
import pandas as pd

# Assuming your DataFrame is named `df`
poetry['label_numeric'] = poetry['label'].map({'CB': 1, 'Non_CB': 0})

# Display the first 5 rows to check the new column
print(poetry[['label', 'label_numeric']].head())

  label  label_numeric
0    CB              1
1    CB              1
2    CB              1
3    CB              1
4    CB              1


In [7]:
poetry.head()

,text,label,cleaned_text,label_numeric
0,"Klo fadli zon goblok, boleeeh?",CB,klo fadli zon goblok boleeeh,1
1,"@detikcom Jangan nyinyirin pak fadli zon, dia ...",CB,jangan nyinyirin pak fadli zon dia gitu punya...,1
2,@ahmadwaluy @bravo1282 @fadlizon Kritik siapa ...,CB,kritik siapa bisa di penjara fadli zon krit...,1
3,"@fadlizon @jokowi Hati penuh dendam, pikiran...",CB,hati penuh dendam pikiran selalu negatif m...,1
4,@fadlizon @jokowi Kerja anda apa zon bisanya c...,CB,kerja anda apa zon bisanya cuma nyinyi r aja...,1


In [8]:
from sklearn.model_selection import train_test_split

X = poetry['cleaned_text']
y = poetry['label_numeric']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)

X_train_vec = vectorizer.fit_transform(X_train)

X_test_vec = vectorizer.transform(X_test)


In [10]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

model.fit(X_train_vec, y_train)


LogisticRegression()

In [11]:
from sklearn.metrics import classification_report, accuracy_score

# Melakukan prediksi pada data pengujian
y_pred = model.predict(X_test_vec)
y_pred_proba = model.predict_proba(X_test_vec)

# Menghitung akurasi
accuracy = accuracy_score(y_test, y_pred)
print(f"Akurasi: {accuracy:.2f}")

# Menampilkan laporan klasifikasi
print(classification_report(y_test, y_pred))


Akurasi: 0.81
              precision    recall  f1-score   support

           0       0.88      0.66      0.75       119
           1       0.79      0.93      0.85       162

    accuracy                           0.81       281
   macro avg       0.83      0.79      0.80       281
weighted avg       0.82      0.81      0.81       281



In [12]:
results = []

for i in range(len(X_test)):
    teks = X_test.iloc[i]
    pred_label = 'CB' if y_pred[i] == 1 else 'Non_CB'
    confidence = y_pred_proba[i][1] if y_pred[i] == 1 else 1 - y_pred_proba[i][1]
    
    results.append({
        'Text': teks,
        'Label': pred_label,
        'Confidence': confidence * 100  
    })

results_df = pd.DataFrame(results)

In [13]:
results_df

,Text,Label,Confidence
0,bapak president terhormat jangan asik berkunj...,Non_CB,60.573184
1,shio baekhyun itu monyet,CB,73.555087
2,caper tolol babi saya tunggu lu di semester ti...,CB,69.000782
3,apa yang harus di banggakan dari polisi merek...,CB,60.338067
4,bajingan bolt hari ini koneksi kenceng tapi me...,CB,56.376623
...,...,...,...
276,anak bajingan ini,CB,71.781789
277,yang gua bingung jujur kenapa media pada rebut...,Non_CB,56.084666
278,begini doang gw jejeritan anjing,CB,72.835604
279,di tes urin semua bu utk menjaring dan men...,Non_CB,66.894662


In [14]:
import pickle

save_path = 'model_logistic_regression.pkl'

with open(save_path, 'wb') as file:
    pickle.dump(model, file)

print(f"Model berhasil disimpan di {save_path}")


Model berhasil disimpan di model_logistic_regression.pkl


In [15]:
import pickle

# Memuat model dari file
with open('model_logistic_regression.pkl', 'rb') as file:
    model = pickle.load(file)

with open('tfidf_vectorizer.pkl', 'wb') as vectorizer_file:
    pickle.dump(vectorizer, vectorizer_file)